<a href="https://colab.research.google.com/github/simonarahi/MachineLearning-TensorFlow/blob/master/latest_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simona Rahi
# Loading Data and Classifying 

In [0]:
import tensorflow as tf
from tensorflow import keras
import functools
import numpy as np
import pandas as pd

Loading test and train data (each is 50% of data)

In [0]:
data_URL = "https://raw.githubusercontent.com/simonarahi/MachineLearning-TensorFlow/master/nassCDS.csv"

train_data_URL = "https://raw.githubusercontent.com/simonarahi/MachineLearning-TensorFlow/master/train.csv"
test_data_URL = "https://raw.githubusercontent.com/simonarahi/MachineLearning-TensorFlow/master/test.csv"

train_file_path = tf.keras.utils.get_file("train.csv", train_data_URL)
test_file_path = tf.keras.utils.get_file("test.csv", test_data_URL)

In [0]:
np.set_printoptions(precision=3, suppress=True)

Inspecting the data

In [0]:
df = pd.read_csv(train_file_path)
df.head()
#df['injSeverity'].min()
#df['injSeverity'].max()

,Unnamed: 0,X,dvcat,weight,dead,airbag,seatbelt,frontal,sex,ageOFocc,yearacc,yearVeh,abcat,occRole,deploy,injSeverity,caseid
0,1,1,25-39,25.069,alive,none,belted,1,f,26,1997,1990.0,unavail,driver,0,3.0,2:3:1
1,3,3,10-24,32.379,alive,none,none,1,f,69,1997,1988.0,unavail,driver,0,4.0,2:5:1
2,6,6,40-54,25.069,alive,none,belted,1,f,22,1997,1985.0,unavail,driver,0,3.0,2:11:2
3,7,7,55+,27.078,alive,none,belted,1,m,22,1997,1984.0,unavail,driver,0,3.0,2:13:1
4,9,9,10-24,812.869,alive,none,belted,0,m,40,1997,1984.0,unavail,driver,0,1.0,2:14:1


Specifying column to be classified

In [0]:
LABEL_COLUMN = 'injSeverity'
LABELS = [0, 1, 2, 3, 4, 5, 6]

Read csv data from file and create dataset

In [0]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      select_columns = ['dvcat', 'weight', 'dead', 'airbag', 'seatbelt', 'frontal', 'sex', 'ageOFocc', 'yearacc', 'yearVeh', 'abcat', 'occRole', 'deploy', 'injSeverity'],
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

show_batch(raw_train_data)

dvcat               : [b'25-39' b'25-39' b'10-24' b'25-39' b'10-24']
weight              : [309.351  13.284 299.272 149.546  63.452]
dead                : [b'alive' b'alive' b'alive' b'alive' b'alive']
airbag              : [b'airbag' b'airbag' b'none' b'none' b'airbag']
seatbelt            : [b'belted' b'none' b'belted' b'belted' b'belted']
frontal             : [1 0 1 0 1]
sex                 : [b'm' b'm' b'f' b'f' b'm']
ageOFocc            : [41 19 19 64 19]
yearacc             : [1998 1998 1998 1998 1998]
yearVeh             : [1996 1999 1983 1992 1998]
abcat               : [b'deploy' b'deploy' b'unavail' b'unavail' b'deploy']
occRole             : [b'driver' b'driver' b'pass' b'driver' b'pass']
deploy              : [1 1 0 0 1]


# **Data Preprocessing**


Since we have mixed data types, we will start by seperating the numeric features and pack them into one column.

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
NUMERIC_FEATURES = ['weight','frontal','ageOFocc', 'yearacc', 'yearVeh', 'deploy']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [0]:
show_batch(packed_train_data)

dvcat               : [b'55+' b'10-24' b'25-39' b'40-54' b'10-24']
dead                : [b'alive' b'alive' b'alive' b'alive' b'alive']
airbag              : [b'none' b'none' b'none' b'airbag' b'none']
seatbelt            : [b'belted' b'belted' b'belted' b'none' b'belted']
sex                 : [b'f' b'm' b'f' b'm' b'm']
abcat               : [b'unavail' b'unavail' b'unavail' b'deploy' b'unavail']
occRole             : [b'driver' b'pass' b'driver' b'driver' b'pass']
caseid              : [b'2:98:1' b'6:47:2' b'45:145:1' b'9:52:1' b'49:278:1']
numeric             : [[   50.163     1.       51.     1998.     1993.        0.   ]
 [ 1885.771     0.       19.     1997.     1991.        0.   ]
 [18856.04      1.       21.     2001.     1988.        0.   ]
 [   35.785     0.       32.     2000.     1994.        1.   ]
 [    9.785     1.       49.     2000.     2000.        0.   ]]


In [0]:
example_batch, labels_batch = next(iter(packed_train_data))

Normalizing our continuous data

In [0]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,weight,frontal,ageOFocc,yearacc,yearVeh,deploy
count,13109.000000,13109.000000,13109.000000,13109.000000,13108.000000,13109.000000
mean,469.761290,0.641315,37.173697,1999.555496,1992.813168,0.340835
std,1609.146959,0.479633,17.879764,1.702573,5.628077,0.474008
min,0.000000,0.000000,16.000000,1997.000000,1953.000000,0.000000
25%,33.185000,0.000000,22.000000,1998.000000,1989.000000,0.000000
50%,86.986000,1.000000,33.000000,2000.000000,1994.000000,0.000000
75%,362.130000,1.000000,48.000000,2001.000000,1997.000000,1.000000
max,57871.595000,1.000000,97.000000,2002.000000,2003.000000,1.000000


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

Creating a numeric column with the normalized data

In [0]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(6,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fa7e03da598>, mean=array([ 469.761,    0.641,   37.174, 1999.555, 1992.813,    0.341]), std=array([1609.147,    0.48 ,   17.88 ,    1.703,    5.628,    0.474])))

We will include this feature column 'numeric' in our training process 

In [0]:
example_batch['numeric']

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[ 555.43 ,    1.   ,   16.   , 2000.   , 1989.   ,    0.   ],
       [  45.52 ,    0.   ,   17.   , 2001.   , 1999.   ,    0.   ],
       [ 198.299,    1.   ,   16.   , 1999.   , 1995.   ,    1.   ],
       [3988.548,    0.   ,   39.   , 1997.   , 1985.   ,    0.   ],
       [  32.356,    1.   ,   42.   , 1998.   , 1978.   ,    0.   ]],
      dtype=float32)>

In [0]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.053,  0.748, -1.184,  0.261, -0.678, -0.719],
       [-0.264, -1.337, -1.128,  0.848,  1.099, -0.719],
       [-0.169,  0.748, -1.184, -0.326,  0.389,  1.391],
       [ 2.187, -1.337,  0.102, -1.501, -1.388, -0.719],
       [-0.272,  0.748,  0.27 , -0.914, -2.632, -0.719]], dtype=float32)

Now dealing with our categorical variables

In [0]:
CATEGORIES = {
    'dvcat': ['1-9km/h', '10-24', '25-39', '40-54', '55+'],
    'dead' : ['alive', 'dead'],
    'airbag' : ['airbag', 'none'],
    'seatbelt' : ['belted', 'none'],
    'sex' : ['f', 'm'],
    'abcat' : ['deploy', 'nodeploy', 'unavail'],
    'occRole' : ['driver', 'pass']
}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='dvcat', vocabulary_list=('1-9km/h', '10-24', '25-39', '40-54', '55+'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='dead', vocabulary_list=('alive', 'dead'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='airbag', vocabulary_list=('airbag', 'none'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='seatbelt', vocabulary_list=('belted', 'none'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('f', 'm'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='abcat', vocabulary_list=('deploy', 'nodeplo

This layer below will be part of the data processing input layer in our model

In [0]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0.]


**Combined Preprocessing Layer:**
Adding the two feature column we created (continuous and categorical) to create an input layer that will extract and preprocess both input types

In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     0.     1.     0.     1.     1.     0.     0.     0.     0.
  1.     0.     0.053  0.748 -1.184  0.261 -0.678 -0.719  0.     1.
  0.     1.     1.     0.   ]


# **Build the Model**

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

now we can start training

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [0]:
model.fit(train_data, epochs=20)

Epoch 1/20
2606/2606 [==============================] - 10s 4ms/step - loss: -621966.3125 - accuracy: 0.2138
Epoch 2/20
2606/2606 [==============================] - 10s 4ms/step - loss: -6858165.5000 - accuracy: 0.2138
Epoch 3/20
2606/2606 [==============================] - 10s 4ms/step - loss: -23168572.0000 - accuracy: 0.2138
Epoch 4/20
2606/2606 [==============================] - 9s 4ms/step - loss: -52472800.0000 - accuracy: 0.2135
Epoch 5/20
2606/2606 [==============================] - 10s 4ms/step - loss: -96877472.0000 - accuracy: 0.2138
Epoch 6/20
2606/2606 [==============================] - 10s 4ms/step - loss: -159050704.0000 - accuracy: 0.2142
Epoch 7/20
2606/2606 [==============================] - 10s 4ms/step - loss: -241686560.0000 - accuracy: 0.2136
Epoch 8/20
2606/2606 [==============================] - 9s 4ms/step - loss: -347958624.0000 - accuracy: 0.2136
Epoch 9/20
2606/2606 [==============================] - 10s 4ms/step - loss: -480158592.0000 - accuracy: 0.2136
Ep

Now that we trained our model, we can test the accuracy on the test set

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

2607/2607 [==============================] - 8s 3ms/step - loss: -4677221888.0000 - accuracy: 0.2173


Test Loss -4677221888.0, Test Accuracy 0.21726122498512268


Test accuracy is almost 21.7% which is very low

In [0]:
predictions = model.predict(test_data)